#  Efficient training with multitask-serving with OpenDelta

### PART 1
In this notebook, you will see three interesting task around a question provided by a user.
1. We load a Delta to correct the spelling in the question.

2. We load another Delta to recognize the question's topic.

3.  We load another Delta to answer the question.

All this functionality shares exactly the same backbone model: T5-large. You can learn how to use the OpenDelta for multitask-serving with 1/N GPU RAM (N is the number of tasks) compared to finetuning.

### PART2 
Then you will learn how to use OpenDelta to train a t5-large using batchsize=32 with only 11G GPU RAM (Which is provided by google for free).
**（You need to connect to the free GPU on the upper-right of the page.)**

### PART3
You will see the flexibility that OpenDelta provides to customize a delta model.

In [ ]:
!nvidia-smi

Mon Jul  4 04:02:04 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
# install necessary packages
!pip install transformers --quiet
!pip install datasets==2.0 --quiet
!pip install opendelta==0.2.2 --quiet

     |████████████████████████████████| 4.4 MB 14.5 MB/s 
     |████████████████████████████████| 101 kB 12.4 MB/s 
     |████████████████████████████████| 596 kB 55.7 MB/s 
     |████████████████████████████████| 6.6 MB 64.6 MB/s 
     |████████████████████████████████| 325 kB 14.3 MB/s 
     |████████████████████████████████| 1.1 MB 71.3 MB/s 
     |████████████████████████████████| 212 kB 67.0 MB/s 
     |████████████████████████████████| 140 kB 71.2 MB/s 
     |████████████████████████████████| 127 kB 75.6 MB/s 
     |████████████████████████████████| 271 kB 75.1 MB/s 
     |████████████████████████████████| 144 kB 79.0 MB/s 
     |████████████████████████████████| 94 kB 4.4 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████|

In [2]:
from dataclasses import dataclass, field
from typing import Optional, List
from transformers import Seq2SeqTrainingArguments, TrainerCallback 
from datasets import load_dataset, load_metric, concatenate_datasets
import transformers
from transformers import (
    AutoConfig,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    HfArgumentParser,
    MBartTokenizer,
    default_data_collator,
    set_seed,
)
from datasets import load_dataset
import torch
import numpy as np
import random

In [3]:
@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune from.
    """
    model_name_or_path: str = field(
        metadata={"help": "Path to pretrained model or model identifier from huggingface.co/models"}
    )
    config_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained config name or path if not the same as model_name"}
    )
    tokenizer_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained tokenizer name or path if not the same as model_name"}
    )
    cache_dir: Optional[str] = field(
        default=None,
        metadata={"help": "Where to store the pretrained models downloaded from huggingface.co"},
    )
    use_fast_tokenizer: bool = field(
        default=True,
        metadata={"help": "Whether to use one of the fast tokenizer (backed by the tokenizers library) or not."},
    )
    model_revision: str = field(
        default="main",
        metadata={"help": "The specific model version to use (can be a branch name, tag name or commit id)."},
    )
    use_auth_token: bool = field(
        default=False,
        metadata={
            "help": "Will use the token generated when running `transformers-cli login` (necessary to use this script "
            "with private models)."
        },
    )

model_args = ModelArguments(model_name_or_path="t5-large", )

### Load the backbone model using the traditional ways.

In [4]:
config = AutoConfig.from_pretrained(
    model_args.config_name if model_args.config_name else model_args.model_name_or_path,
    cache_dir=model_args.cache_dir,
    revision=model_args.model_revision,
    use_auth_token=True if model_args.use_auth_token else None,
)
config.dropout_rate = 0.0
tokenizer = AutoTokenizer.from_pretrained(
    model_args.tokenizer_name if model_args.tokenizer_name else model_args.model_name_or_path,
    cache_dir=model_args.cache_dir,
    use_fast=model_args.use_fast_tokenizer,
    revision=model_args.model_revision,
    use_auth_token=True if model_args.use_auth_token else None,
)
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_args.model_name_or_path,
    from_tf=bool(".ckpt" in model_args.model_name_or_path),
    config=config,
    cache_dir=model_args.cache_dir,
    revision=model_args.model_revision,
    use_auth_token=True if model_args.use_auth_token else None,
)
model.resize_token_embeddings(len(tokenizer))


Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5_fast.py:166: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


Downloading:   0%|          | 0.00/2.75G [00:00<?, ?B/s]

Embedding(32100, 1024)

### PART0: Visualization
#### Using OpenDelta as a fantastic visualization tool.

In [5]:
from opendelta import Visualization
Visualization(model).structure_graph();

root
├── shared(Embedding),lm_head(Linear) weight:[32100, 1024]
├── encoder (T5Stack)
│   ├── embed_tokens (Embedding) weight:[32100, 1024]
│   ├── block (ModuleList)
│   │   ├── 0 (T5Block)
│   │   │   └── layer (ModuleList)
│   │   │       ├── 0 (T5LayerSelfAttention)
│   │   │       │   ├── SelfAttention (T5Attention)
│   │   │       │   │   ├── q,k,v,o(Linear) weight:[1024, 1024]
│   │   │       │   │   └── relative_attention_bias (Embedding) weight:[32, 16]
│   │   │       │   └── layer_norm (T5LayerNorm) weight:[1024]
│   │   │       └── 1 (T5LayerFF)
│   │   │           ├── DenseReluDense (T5DenseActDense)
│   │   │           │   ├── wi (Linear) weight:[4096, 1024]
│   │   │           │   └── wo (Linear) weight:[1024, 4096]
│   │   │           └── layer_norm (T5LayerNorm) weight:[1024]
│   │   └── 1-23(T5Block)
│   │       └── layer (ModuleList)
│   │           ├── 0 (T5LayerSelfAttention)
│   │           │   ├── SelfAttention (T5Attention)
│   │           │   │   └── q,k,v,o(Linear) weight:[1024, 1024]
│   │           │   └── layer_norm (T5LayerNorm) weight:[1024]
│   │           └── 1 (T5LayerFF)
│   │               ├── DenseReluDense (T5DenseActDense)
│   │               │   ├── wi (Linear) weight:[4096, 1024]
│   │               │   └── wo (Linear) weight:[1024, 4096]
│   │               └── layer_norm (T5LayerNorm) weight:[1024]
│   └── final_layer_norm (T5LayerNorm) weight:[1024]
└── decoder (T5Stack)
    ├── embed_tokens (Embedding) weight:[32100, 1024]
    ├── block (ModuleList)
    │   ├── 0 (T5Block)
    │   │   └── layer (ModuleList)
    │   │       ├── 0 (T5LayerSelfAttention)
    │   │       │   ├── SelfAttention (T5Attention)
    │   │       │   │   ├── q,k,v,o(Linear) weight:[1024, 1024]
    │   │       │   │   └── relative_attention_bias (Embedding) weight:[32, 16]
    │   │       │   └── layer_norm (T5LayerNorm) weight:[1024]
    │   │       ├── 1 (T5LayerCrossAttention)
    │   │       │   ├── EncDecAttention (T5Attention)
    │   │       │   │   └── q,k,v,o(Linear) weight:[1024, 1024]
    │   │       │   └── layer_norm (T5LayerNorm) weight:[1024]
    │   │       └── 2 (T5LayerFF)
    │   │           ├── DenseReluDense (T5DenseActDense)
    │   │           │   ├── wi (Linear) weight:[4096, 1024]
    │   │           │   └── wo (Linear) weight:[1024, 4096]
    │   │           └── layer_norm (T5LayerNorm) weight:[1024]
    │   └── 1-23(T5Block)
    │       └── layer (ModuleList)
    │           ├── 0 (T5LayerSelfAttention)
    │           │   ├── SelfAttention (T5Attention)
    │           │   │   └── q,k,v,o(Linear) weight:[1024, 1024]
    │           │   └── layer_norm (T5LayerNorm) weight:[1024]
    │           ├── 1 (T5LayerCrossAttention)
    │           │   ├── EncDecAttention (T5Attention)
    │           │   │   └── q,k,v,o(Linear) weight:[1024, 1024]
    │           │   └── layer_norm (T5LayerNorm) weight:[1024]
    │           └── 2 (T5LayerFF)
    │               ├── DenseReluDense (T5DenseActDense)
    │               │   ├── wi (Linear) weight:[4096, 1024]
    │               │   └── wo (Linear) weight:[1024, 4096]
    │               └── layer_norm (T5LayerNorm) weight:[1024]
    └── final_layer_norm (T5LayerNorm) weight:[1024]

#### Compare to the original visualization in pytorch

In [ ]:
model

### PART1:  Multitask Serving with Finetuned Deltas.
Load the three special delta models with different functionalities.

In [6]:
from opendelta import AutoDeltaConfig, AutoDeltaModel

delta_model_spelling = AutoDeltaModel.from_finetuned("thunlp/Spelling_Correction_T5_LRAdapter_demo", backbone_model=model)
delta_model_spelling.detach()

delta_model_topic = AutoDeltaModel.from_finetuned("thunlp/Question_Topic_T5-large_Compacter", backbone_model=model)
delta_model_topic.detach()

delta_model_fact = AutoDeltaModel.from_finetuned("thunlp/FactQA_T5-large_Adapter", backbone_model=model)
delta_model_fact.detach()


100% Download Spelling_Correction_T5_LRAdapter_demo.zip 

[WARNING|(OpenDelta)delta_configs:195]2022-07-23 01:26:38,925 >> The following keys are not used by <class 'opendelta.delta_models.low_rank_adapter.LowRankAdapterConfig'>.__init__ function: ['delta_type', 'opendelta_version', 'transformers_version']
[INFO|(OpenDelta)delta_configs:209]2022-07-23 01:26:38,927 >> Model config
LowRankAdapterConfig {
  "backbone_checkpoint_name": "t5-large",
  "backbone_class": "T5ForConditionalGeneration",
  "backbone_hash": "baa014d5e81363ff48935ba78b5df374",
  "common_structure": null,
  "low_rank_rank": 1,
  "low_rank_w_init": "glorot-uniform",
  "modified_modules": [
    "SelfAttention",
    "DenseReluDense"
  ],
  "non_linearity": "gelu_new",
  "opendelta_version": "0.2.2",
  "reduction_factor": 32,
  "transformers_version": "4.20.1"
}



Reuse the cached checkpoint in /root/.cache/delta_center/Spelling_Correction_T5_LRAdapter_demo
[WARNING|(OpenDelta)saving_loading_utils:285]2022-07-23 01:26:49,901 >> The config has an hash of the backbone model, and isdifferent from the hash of the loaded model. This indicates a mismatchbetween the backbone model that the delta checkpoint is based on andthe one you loaded. You propobability need to Train the model instead ofdirectly inference. 


100% Download Question_Topic_T5-large_Compacter.zip 


[WARNING|(OpenDelta)delta_configs:195]2022-07-23 01:26:53,067 >> The following keys are not used by <class 'opendelta.delta_models.compacter.CompacterConfig'>.__init__ function: ['delta_type', 'opendelta_version', 'transformers_version']
[INFO|(OpenDelta)delta_configs:209]2022-07-23 01:26:53,073 >> Model config
CompacterConfig {
  "backbone_checkpoint_name": "t5-large",
  "backbone_class": "T5ForConditionalGeneration",
  "backbone_hash": "6297bd1acc36524547c8a76cc03fef5c",
  "bottleneck_dim": null,
  "common_structure": null,
  "factorized_phm": true,
  "factorized_phm_rule": false,
  "hypercomplex_division": 4,
  "hypercomplex_nonlinearity": "glorot-uniform",
  "kronecker_prod": null,
  "learn_phm": true,
  "modified_modules": [
    "SelfAttention",
    "DenseReluDense"
  ],
  "non_linearity": "gelu_new",
  "opendelta_version": "0.2.2",
  "phm_c_init": "normal",
  "phm_init_range": 0.0001,
  "phm_rank": 1,
  "reduction_factor": 16,
  "sequential": null,
  "shared_W_phm": false,
  "sha

100% Download FactQA_T5-large_Adapter.zip 


[WARNING|(OpenDelta)delta_configs:195]2022-07-23 01:27:09,292 >> The following keys are not used by <class 'opendelta.delta_models.adapter.AdapterConfig'>.__init__ function: ['delta_type', 'opendelta_version', 'transformers_version']
[INFO|(OpenDelta)delta_configs:209]2022-07-23 01:27:09,298 >> Model config
AdapterConfig {
  "backbone_checkpoint_name": "t5-large",
  "backbone_class": "T5ForConditionalGeneration",
  "backbone_hash": "a984db8cc3fe15bfb2618b1ff7abd570",
  "bottleneck_dim": 24,
  "common_structure": null,
  "modified_modules": [
    "SelfAttention",
    "DenseReluDense"
  ],
  "non_linearity": "gelu_new",
  "opendelta_version": "0.2.2",
  "sequential": true,
  "transformers_version": "4.20.1"
}

Reuse the cached checkpoint in /root/.cache/delta_center/FactQA_T5-large_Adapter
[WARNING|(OpenDelta)saving_loading_utils:285]2022-07-23 01:27:20,450 >> The config has an hash of the backbone model, and isdifferent from the hash of the loaded model. This indicates a mismatchbetween

In [7]:
def multitask_serving(input_text):
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids#.cuda()
    delta_model_spelling.attach()
    answers_ids =model.generate(input_ids=input_ids, max_length=20, num_beams=4)
    input_text = tokenizer.decode(answers_ids[0], skip_special_tokens=True)
    print("Correct Spelling: {}".format(input_text))
    delta_model_spelling.detach()

    delta_model_topic.attach()
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids#.cuda()
    answers_ids =model.generate(input_ids=input_ids, max_length=20, num_beams=4)
    topic = tokenizer.decode(answers_ids[0], skip_special_tokens=True)
    delta_model_topic.detach()
    print("Question Topic: {}".format(topic))

    delta_model_fact.attach()
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids#.cuda()
    answers_ids =model.generate(input_ids=input_ids, max_length=20, num_beams=4)
    input_text = tokenizer.decode(answers_ids[0], skip_special_tokens=True)
    delta_model_fact.detach()
    print("Question Answer: {}".format(input_text))




In [8]:
multitask_serving("When was Beiiing olymp#ic heldd ?")
multitask_serving("What the commmon career of Newton ad eintesin?")

Correct Spelling: When was Beijing Olympic held?
Question Topic: The question's topic is sports.
Question Answer: 2008
Correct Spelling: What was the common career of Newton and Einstein?
Question Topic: The question's topic is science.
Question Answer: Physicists


In [9]:
delta_model_spelling.detach()
delta_model_topic.detach()
delta_model_fact.detach()

## PART2: Efficient Tuning a T5-large with Limited GPU RAM

###  spelling correction as an example

In [10]:
@dataclass
class TrainingArguments(Seq2SeqTrainingArguments):
    print_num_parameters: Optional[bool] = field(default=False, metadata={"help": "If set, print the parameters of "
                                                                                 "the model."})
    do_test: Optional[bool] = field(default=False, metadata={"help": "If set, evaluates the test performance."})
    split_validation_test: Optional[bool] = field(default=False,
                                                  metadata={"help": "If set, for the datasets which do not"
                                                                    "have the test set, we use validation set as their"
                                                                    "test set and make a validation set from either"
                                                                    "splitting the validation set into half (for smaller"
                                                                    "than 10K samples datasets), or by using 1K examples"
                                                                    "from training set as validation set (for larger"
                                                                    " datasets)."})
    compute_time: Optional[bool] = field(default=False, metadata={"help": "If set measures the time."})
    compute_memory: Optional[bool] = field(default=False, metadata={"help": "if set, measures the memory"})


training_args = TrainingArguments(output_dir="./", 
                                  do_train=True,
                                  do_eval=True,
                                  do_predict=False,
                                  evaluation_strategy="steps",
                                  eval_steps=200,
                                  save_strategy="steps",
                                  save_steps=200,
                                  greater_is_better=True,
                                  load_best_model_at_end=True,
                                  compute_memory=True,
                                  predict_with_generate=True,
                                  push_to_hub=False,
                                  learning_rate=1e-3,
                                  seed=42,
                                  per_device_eval_batch_size=32,
                                  per_device_train_batch_size=32,
                                  num_train_epochs=1,
                                  metric_for_best_model="em",
                                  warmup_steps=0,
                                  save_total_limit=1,
                                  gradient_accumulation_steps=1
                                  )

In [11]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    decoded_preds = [tokenizer.decode(i, skip_special_tokens=True, clean_up_tokenization_spaces=True) for i in preds]
    decoded_labels = [tokenizer.decode(i, skip_special_tokens=True, clean_up_tokenization_spaces=True) for i in labels]
    result = {}
    result_list = [int(i==j) for i, j in zip(decoded_labels, decoded_preds)]
    result.update({"em":sum(result_list)/len(result_list)})
    return result

In [12]:
mydataset = load_dataset("trivia_qa","unfiltered.nocontext")
mydataset['train'] = mydataset['train']
validation_index = np.arange(len(mydataset['validation']))
np.random.shuffle(validation_index)
mydataset['validation'] = mydataset['validation'].select(validation_index[:500])

Downloading:   0%|          | 0.00/3.70k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.99k [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/633M [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset trivia_qa downloaded and prepared to /root/.cache/huggingface/datasets/trivia_qa/unfiltered.nocontext/1.2.0/e73c5e47a8704744fa9ded33504b35a6c098661813d1c2a09892eb9b9e9d59ae. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

We randomly disturb the input question, for example change "Is it a good movie?" to "Ia it af movi good?" The target is the original sentences. 

In [13]:
def misspelling(x):
    length = len(x)
    replace_time = np.random.randint(3)
    count = 0

    while (count<replace_time):
        randfloat = np.random.rand()
        if randfloat < 0.15:
            x = x.split()
            switch_index = [np.random.randint(low=0, high=len(x)) for i in range(2)]
            tmp = x[switch_index[0]]
            x[switch_index[0]] = x[switch_index[1]]
            x[switch_index[1]] = tmp
            x = " ".join(x)
        elif randfloat < 0.3:
            x = x.split()
            drop_index = np.random.randint(low=0, high=len(x))
            x = x[:drop_index] + x[drop_index+1:]
            x = " ".join(x)
        elif randfloat < 0.8:
            replace_str = "".join([random.choice('abcdefghijklmnopqrstuvwxyz!@#$%^&*()') for i in range(np.random.randint(1,3))])
            rindx = np.random.randint(low=0, high=length)
            x = x[:rindx]+replace_str+x[rindx+1:]
        else:
            x=list(x)
            switch_index = [np.random.randint(low=0, high=len(x)) for i in range(2)]
            tmp = x[switch_index[0]]
            x[switch_index[0]] = x[switch_index[1]]
            x[switch_index[1]] = tmp
            x = "".join(x)
        count+=1
    return x

def tokenize_function(examples):
    input_sentences = [" ".join((i.strip("\n").strip().strip("?")+"?").split()[:20]) for i in examples["question"]]
    mis_spellings = [misspelling(x) for x in input_sentences]
    input_ids = [tokenizer.encode(i, padding="max_length", truncation=True, max_length=64) for i in mis_spellings]
    label = [tokenizer.encode(i, padding="max_length", truncation=True, max_length=64) for i in input_sentences]
    return {"input_ids": input_ids, "labels": label}

tokenized_datasets = mydataset.map(tokenize_function, remove_columns=['answer', 'question_source',"entity_pages",'search_results'], batched=True)

Parameter 'function'=<function tokenize_function at 0x7faa8c016560> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/88 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/11 [00:00<?, ?ba/s]

### Now let's apply a LowRankAdapterModel

In [14]:
from opendelta import LowRankAdapterModel
delta_model1 = LowRankAdapterModel(backbone_model=model, modified_modules=['SelfAttention', "DenseReluDense"])
delta_model1.freeze_module(set_state_dict = True)
delta_model1.log(delta_ratio=True, trainable_ratio=True, visualization=True)


root
├── shared(Embedding),lm_head(Linear) weight:[32100, 1024]
├── encoder (T5Stack)
│   ├── embed_tokens (Embedding) weight:[32100, 1024]
│   ├── block (ModuleList)
│   │   ├── 0 (T5Block)
│   │   │   └── layer (ModuleList)
│   │   │       ├── 0 (T5LayerSelfAttention)
│   │   │       │   ├── SelfAttention (T5Attention)
│   │   │       │   │   ├── q,k,v,o(Linear) weight:[1024, 1024]
│   │   │       │   │   ├── relative_attention_bias (Embedding) weight:[32, 16]
│   │   │       │   │   └── low_rank_adapter_1 (LowRankAdapter)
│   │   │       │   │       ├── down_sampler (LowRankLinear) W_left:[1024, 1] W_right:[1, 32] b:[32]
│   │   │       │   │       └── up_sampler (LowRankLinear) W_left:[32, 1] W_right:[1, 1024] b:[1024]
│   │   │       │   └── layer_norm (T5LayerNorm) weight:[1024]
│   │   │       └── 1 (T5LayerFF)
│   │   │           ├── DenseReluDense (T5DenseActDense)
│   │   │           │   ├── wi (Linear) weight:[4096, 1024]
│   │   │           │   ├── wo (Linear) weight:[1024, 4096]
│   │   │           │   └── low_rank_adapter_1 (LowRankAdapter)
│   │   │           │       ├── down_sampler (LowRankLinear) W_left:[1024, 1] W_right:[1, 32] b:[32]
│   │   │           │       └── up_sampler (LowRankLinear) W_left:[32, 1] W_right:[1, 1024] b:[1024]
│   │   │           └── layer_norm (T5LayerNorm) weight:[1024]
│   │   └── 1-23(T5Block)
│   │       └── layer (ModuleList)
│   │           ├── 0 (T5LayerSelfAttention)
│   │           │   ├── SelfAttention (T5Attention)
│   │           │   │   ├── q,k,v,o(Linear) weight:[1024, 1024]
│   │           │   │   └── low_rank_adapter_1 (LowRankAdapter)
│   │           │   │       ├── down_sampler (LowRankLinear) W_left:[1024, 1] W_right:[1, 32] b:[32]
│   │           │   │       └── up_sampler (LowRankLinear) W_left:[32, 1] W_right:[1, 1024] b:[1024]
│   │           │   └── layer_norm (T5LayerNorm) weight:[1024]
│   │           └── 1 (T5LayerFF)
│   │               ├── DenseReluDense (T5DenseActDense)
│   │               │   ├── wi (Linear) weight:[4096, 1024]
│   │               │   ├── wo (Linear) weight:[1024, 4096]
│   │               │   └── low_rank_adapter_1 (LowRankAdapter)
│   │               │       ├── down_sampler (LowRankLinear) W_left:[1024, 1] W_right:[1, 32] b:[32]
│   │               │       └── up_sampler (LowRankLinear) W_left:[32, 1] W_right:[1, 1024] b:[1024]
│   │               └── layer_norm (T5LayerNorm) weight:[1024]
│   └── final_layer_norm (T5LayerNorm) weight:[1024]
└── decoder (T5Stack)
    ├── embed_tokens (Embedding) weight:[32100, 1024]
    ├── block (ModuleList)
    │   ├── 0 (T5Block)
    │   │   └── layer (ModuleList)
    │   │       ├── 0 (T5LayerSelfAttention)
    │   │       │   ├── SelfAttention (T5Attention)
    │   │       │   │   ├── q,k,v,o(Linear) weight:[1024, 1024]
    │   │       │   │   ├── relative_attention_bias (Embedding) weight:[32, 16]
    │   │       │   │   └── low_rank_adapter_1 (LowRankAdapter)
    │   │       │   │       ├── down_sampler (LowRankLinear) W_left:[1024, 1] W_right:[1, 32] b:[32]
    │   │       │   │       └── up_sampler (LowRankLinear) W_left:[32, 1] W_right:[1, 1024] b:[1024]
    │   │       │   └── layer_norm (T5LayerNorm) weight:[1024]
    │   │       ├── 1 (T5LayerCrossAttention)
    │   │       │   ├── EncDecAttention (T5Attention)
    │   │       │   │   └── q,k,v,o(Linear) weight:[1024, 1024]
    │   │       │   └── layer_norm (T5LayerNorm) weight:[1024]
    │   │       └── 2 (T5LayerFF)
    │   │           ├── DenseReluDense (T5DenseActDense)
    │   │           │   ├── wi (Linear) weight:[4096, 1024]
    │   │           │   ├── wo (Linear) weight:[1024, 4096]
    │   │           │   └── low_rank_adapter_1 (LowRankAdapter)
    │   │           │       ├── down_sampler (LowRankLinear) W_left:[1024, 1] W_right:[1, 32] b:[32]
    │   │           │       └── up_sampler (LowRankLinear) W_left:[32, 1] W_right:[1, 1024] b:[1024]
    │   │           └── layer_norm (T5LayerNorm) weight:[1024]
    │   └── 1-23(T5Bl

[INFO|(OpenDelta)basemodel:675]2022-07-23 01:38:50,695 >> Trainable Ratio: 0.041213%
[INFO|(OpenDelta)basemodel:677]2022-07-23 01:38:50,704 >> Delta Parameter Ratio: 0.041213%
[INFO|(OpenDelta)basemodel:679]2022-07-23 01:38:50,705 >> Static Memory 0.00 GB, Max Memory 0.00 GB


In [15]:
class MyCallback(TrainerCallback):
    def on_evaluate(self, args, state, control, **kwargs):
        """
        Event called after an evaluation phase.
        """
        sents = ["was Wher Newton bon?", 
                 "In year which Beiiing Olmpic ld?"
                ]
        for sent in sents:
            input_ids = tokenizer(sent, return_tensors="pt").input_ids.cuda()
            answers_ids =model.generate(input_ids=input_ids, 
                                    max_length=20, 
                                    num_beams=4, 
                                    )
            print("{} {}".format(sent, tokenizer.decode(answers_ids[0], skip_special_tokens=True)))
        print("max allocated memory {} GB".format(torch.cuda.max_memory_allocated(f"cuda:0")/1024**3))
        
from transformers import Seq2SeqTrainer
training_args.output_dir = "./SpellingCorrection" # to avoid conflict
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    callbacks=[MyCallback],
    compute_metrics=compute_metrics,
)

trainer.train()

# delta_model.save_finetuned("Spelling_T5-lowrankadapter", push_to_hub=True) 
# Since we are in a notebook without login in huggingface model hub, we omit this step.

The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: question_id, question. If question_id, question are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 87622
  Num Epochs = 1
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 2739


Step,Training Loss,Validation Loss


KeyboardInterrupt: ignored

## PART3: Examples of Customized Deltas

Here we add Adapters to 0\~5 layers' attention module, and 10\~12 layers' feed-forward module (after layer_norm).

We then add Lora to 0~9 layers' attention's Q matrix.

In [16]:

from opendelta import AdapterModel
from opendelta import LoraModel
delta_model1.detach()
model.to("cpu")
delta_model_custom1 = AdapterModel(backbone_model=model, 
                                   modified_modules=['[r][0-5]\.layer\.0\.SelfAttention', '[r](10|11|12)\.layer\.1\.layer_norm'])
delta_model_custom2 = LoraModel(backbone_model=model, modified_modules=['[r][0-9]\.layer\.0\.SelfAttention\.q'])
delta_model_custom2.freeze_module(set_state_dict = True)
delta_model_custom2.log(delta_ratio=True, trainable_ratio=True, visualization=True)

# delta_model_custom2.detach()

root
├── shared(Embedding),lm_head(Linear) weight:[32100, 1024]
├── encoder (T5Stack)
│   ├── embed_tokens (Embedding) weight:[32100, 1024]
│   ├── block (ModuleList)
│   │   ├── 0 (T5Block)
│   │   │   └── layer (ModuleList)
│   │   │       ├── 0 (T5LayerSelfAttention)
│   │   │       │   ├── SelfAttention (T5Attention)
│   │   │       │   │   ├── q (Linear) weight:[1024, 1024]
│   │   │       │   │   │   └── lora (LowRankLinear) lora_A:[8, 1024] lora_B:[1024, 8]
│   │   │       │   │   ├── k,v,o(Linear) weight:[1024, 1024]
│   │   │       │   │   ├── relative_attention_bias (Embedding) weight:[32, 16]
│   │   │       │   │   └── adapter_1 (AdapterLayer)
│   │   │       │   │       └── modulelist (Sequential)
│   │   │       │   │           ├── down_proj (Linear) weight:[24, 1024] bias:[24]
│   │   │       │   │           └── up_proj (Linear) weight:[1024, 24] bias:[1024]
│   │   │       │   └── layer_norm (T5LayerNorm) weight:[1024]
│   │   │       └── 1 (T5LayerFF)
│   │   │           ├── DenseReluDense (T5DenseActDense)
│   │   │           │   ├── wi (Linear) weight:[4096, 1024]
│   │   │           │   └── wo (Linear) weight:[1024, 4096]
│   │   │           └── layer_norm (T5LayerNorm) weight:[1024]
│   │   ├── 1-5(T5Block)
│   │   │   └── layer (ModuleList)
│   │   │       ├── 0 (T5LayerSelfAttention)
│   │   │       │   ├── SelfAttention (T5Attention)
│   │   │       │   │   ├── q (Linear) weight:[1024, 1024]
│   │   │       │   │   │   └── lora (LowRankLinear) lora_A:[8, 1024] lora_B:[1024, 8]
│   │   │       │   │   ├── k,v,o(Linear) weight:[1024, 1024]
│   │   │       │   │   └── adapter_1 (AdapterLayer)
│   │   │       │   │       └── modulelist (Sequential)
│   │   │       │   │           ├── down_proj (Linear) weight:[24, 1024] bias:[24]
│   │   │       │   │           └── up_proj (Linear) weight:[1024, 24] bias:[1024]
│   │   │       │   └── layer_norm (T5LayerNorm) weight:[1024]
│   │   │       └── 1 (T5LayerFF)
│   │   │           ├── DenseReluDense (T5DenseActDense)
│   │   │           │   ├── wi (Linear) weight:[4096, 1024]
│   │   │           │   └── wo (Linear) weight:[1024, 4096]
│   │   │           └── layer_norm (T5LayerNorm) weight:[1024]
│   │   ├── 6-9(T5Block)
│   │   │   └── layer (ModuleList)
│   │   │       ├── 0 (T5LayerSelfAttention)
│   │   │       │   ├── SelfAttention (T5Attention)
│   │   │       │   │   ├── q (Linear) weight:[1024, 1024]
│   │   │       │   │   │   └── lora (LowRankLinear) lora_A:[8, 1024] lora_B:[1024, 8]
│   │   │       │   │   └── k,v,o(Linear) weight:[1024, 1024]
│   │   │       │   └── layer_norm (T5LayerNorm) weight:[1024]
│   │   │       └── 1 (T5LayerFF)
│   │   │           ├── DenseReluDense (T5DenseActDense)
│   │   │           │   ├── wi (Linear) weight:[4096, 1024]
│   │   │           │   └── wo (Linear) weight:[1024, 4096]
│   │   │           └── layer_norm (T5LayerNorm) weight:[1024]
│   │   ├── 10-12(T5Block)
│   │   │   └── layer (ModuleList)
│   │   │       ├── 0 (T5LayerSelfAttention)
│   │   │       │   ├── SelfAttention (T5Attention)
│   │   │       │   │   └── q,k,v,o(Linear) weight:[1024, 1024]
│   │   │       │   └── layer_norm (T5LayerNorm) weight:[1024]
│   │   │       └── 1 (T5LayerFF)
│   │   │           ├── DenseReluDense (T5DenseActDense)
│   │   │           │   ├── wi (Linear) weight:[4096, 1024]
│   │   │           │   └── wo (Linear) weight:[1024, 4096]
│   │   │           └── layer_norm (T5LayerNorm) weight:[1024]
│   │   │               └── adapter (AdapterLayer)
│   │   │                   └── modulelist (Sequential)
│   │   │                       ├── down_proj (Linear) weight:[24, 1024] bias:[24]
│   │   │                       └── up_proj (Linear) weight:[1024, 24] bias:[1024]
│   │   └── 13-23(T5Block)
│   │       └── layer (ModuleList)
│   │           ├── 0 (T5LayerSelfAttention)
│   │           │   ├── SelfAttention (T5Attention)
│   │           │   │   └── q,k,v,o(Linear) weight:[1024, 1024]
│   │           │   └── layer_norm (T5Laye

[INFO|(OpenDelta)basemodel:675]2022-07-23 01:54:54,829 >> Trainable Ratio: 0.166643%
[INFO|(OpenDelta)basemodel:677]2022-07-23 01:54:54,829 >> Delta Parameter Ratio: 0.166643%
[INFO|(OpenDelta)basemodel:679]2022-07-23 01:54:54,829 >> Static Memory 0.00 GB, Max Memory 9.53 GB


## OPenDelta（git_example）


In [1]:
!pip install transformers
!pip install opendelta


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 14.5 MB/s 
     |████████████████████████████████| 6.6 MB 54.0 MB/s 
     |████████████████████████████████| 596 kB 65.6 MB/s 
     |████████████████████████████████| 101 kB 13.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 80 kB 4.4 MB/s 
     |████████████████████████████████| 235 kB 16.5 MB/s 
     |████████████████████████████████| 181 kB 65.5 MB/s 
     |████████████████████████████████| 306 kB 48.6 MB/s 
     |████████████████████████████████| 623 kB 73.6 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |█

In [2]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("facebook/bart-base") # suppose we load BART

Downloading:   0%|          | 0.00/1.68k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/532M [00:00<?, ?B/s]

Some weights of BartForSequenceClassification were not initialized from the model checkpoint at facebook/bart-base and are newly initialized: ['classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.dense.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
from opendelta import Visualization
Visualization(model).structure_graph()

root
├── model (BartModel)
│   ├── shared (Embedding) weight:[50265, 768]
│   ├── encoder (BartEncoder)
│   │   ├── embed_tokens (Embedding) weight:[50265, 768]
│   │   ├── embed_positions (BartLearnedPositionalEmbedding) weight:[1026, 768]
│   │   ├── layers (ModuleList)
│   │   │   └── 0-5(BartEncoderLayer)
│   │   │       ├── self_attn (BartAttention)
│   │   │       │   └── k_proj,v_proj,q_proj,out_proj(Linear) weight:[768, 768] bias:[768]
│   │   │       ├── self_attn_layer_norm,final_layer_norm(LayerNorm) weight:[768] bias:[768]
│   │   │       ├── fc1 (Linear) weight:[3072, 768] bias:[3072]
│   │   │       └── fc2 (Linear) weight:[768, 3072] bias:[768]
│   │   └── layernorm_embedding (LayerNorm) weight:[768] bias:[768]
│   └── decoder (BartDecoder)
│       ├── embed_tokens (Embedding) weight:[50265, 768]
│       ├── embed_positions (BartLearnedPositionalEmbedding) weight:[1026, 768]
│       ├── layers (ModuleList)
│       │   └── 0-5(BartDecoderLayer)
│       │       ├── self_attn,encoder_attn(BartAttention)
│       │       │   └── k_proj,v_proj,q_proj,out_proj(Linear) weight:[768, 768] bias:[768]
│       │       ├── self_attn_layer_norm,encoder_attn_layer_norm,final_layer_norm(LayerNorm) weight:[768] 
│       │       │   bias:[768]
│       │       ├── fc1 (Linear) weight:[3072, 768] bias:[3072]
│       │       └── fc2 (Linear) weight:[768, 3072] bias:[768]
│       └── layernorm_embedding (LayerNorm) weight:[768] bias:[768]
└── classification_head (BartClassificationHead)
    ├── dense (Linear) weight:[768, 768] bias:[768]
    └── out_proj (Linear) weight:[3, 768] bias:[3]

root
├── model (BartModel)
│   ├── shared (Embedding) weight:[50265, 768]
│   ├── encoder (BartEncoder)
│   │   ├── embed_tokens (Embedding) weight:[50265, 768]
│   │   ├── embed_positions (BartLearnedPositionalEmbedding) weight:[1026, 768]
│   │   ├── layers (ModuleList)
│   │   │   └── 0-5(BartEncoderLayer)
│   │   │       ├── self_attn (BartAttention)
│   │   │       │   └── k_proj,v_proj,q_proj,out_proj(Linear) weight:[768, 768] bias:[768]
│   │   │       ├── self_attn_layer_norm,final_layer_norm(LayerNorm) weight:[768] bias:[768]
│   │   │       ├── fc1 (Linear) weight:[3072, 768] bias:[3072]
│   │   │       └── fc2 (Linear) weight:[768, 3072] bias:[768]
│   │   └── layernorm_embedding (LayerNorm) weight:[768] bias:[768]
│   └── decoder (BartDecoder)
│       ├── embed_tokens (Embedding) weight:[50265, 768]
│       ├── embed_positions (BartLearnedPositionalEmbedding) weight:[1026, 768]
│       ├── layers (ModuleList)
│       │   └── 0-5(BartDecoderLayer)
│       │       ├── self_attn,encoder_attn(BartAttention)
│       │       │   └── k_proj,v_proj,q_proj,out_proj(Linear) weight:[768, 768] bias:[768]
│       │       ├── self_attn_layer_norm,encoder_attn_layer_norm,final_layer_norm(LayerNorm) weight:[768] 
│       │       │   bias:[768]
│       │       ├── fc1 (Linear) weight:[3072, 768] bias:[3072]
│       │       └── fc2 (Linear) weight:[768, 3072] bias:[768]
│       └── layernorm_embedding (LayerNorm) weight:[768] bias:[768]
└── classification_head (BartClassificationHead)
    ├── dense (Linear) weight:[768, 768] bias:[768]
    └── out_proj (Linear) weight:[3, 768] bias:[3]

In [4]:
from opendelta import AdapterModel
delta_model = AdapterModel(backbone_model=model, modified_modules=['fc2'], bottleneck_dim=12)
delta_model.log() # This will visualize the backbone after modification and other information.

root
├── model (BartModel)
│   ├── shared (Embedding) weight:[50265, 768]
│   ├── encoder (BartEncoder)
│   │   ├── embed_tokens (Embedding) weight:[50265, 768]
│   │   ├── embed_positions (BartLearnedPositionalEmbedding) weight:[1026, 768]
│   │   ├── layers (ModuleList)
│   │   │   └── 0-5(BartEncoderLayer)
│   │   │       ├── self_attn (BartAttention)
│   │   │       │   └── k_proj,v_proj,q_proj,out_proj(Linear) weight:[768, 768] bias:[768]
│   │   │       ├── self_attn_layer_norm,final_layer_norm(LayerNorm) weight:[768] bias:[768]
│   │   │       ├── fc1 (Linear) weight:[3072, 768] bias:[3072]
│   │   │       └── fc2 (Linear) weight:[768, 3072] bias:[768]
│   │   │           └── adapter (AdapterLayer)
│   │   │               └── modulelist (Sequential)
│   │   │                   ├── down_proj (Linear) weight:[12, 768] bias:[12]
│   │   │                   └── up_proj (Linear) weight:[768, 12] bias:[768]
│   │   └── layernorm_embedding (LayerNorm) weight:[768] bias:[768]
│   └── decoder (BartDecoder)
│       ├── embed_tokens (Embedding) weight:[50265, 768]
│       ├── embed_positions (BartLearnedPositionalEmbedding) weight:[1026, 768]
│       ├── layers (ModuleList)
│       │   └── 0-5(BartDecoderLayer)
│       │       ├── self_attn,encoder_attn(BartAttention)
│       │       │   └── k_proj,v_proj,q_proj,out_proj(Linear) weight:[768, 768] bias:[768]
│       │       ├── self_attn_layer_norm,encoder_attn_layer_norm,final_layer_norm(LayerNorm) weight:[768] 
│       │       │   bias:[768]
│       │       ├── fc1 (Linear) weight:[3072, 768] bias:[3072]
│       │       └── fc2 (Linear) weight:[768, 3072] bias:[768]
│       │           └── adapter (AdapterLayer)
│       │               └── modulelist (Sequential)
│       │                   ├── down_proj (Linear) weight:[12, 768] bias:[12]
│       │                   └── up_proj (Linear) weight:[768, 12] bias:[768]
│       └── layernorm_embedding (LayerNorm) weight:[768] bias:[768]
└── classification_head (BartClassificationHead)
    ├── dense (Linear) weight:[768, 768] bias:[768]
    └── out_proj (Linear) weight:[3, 768] bias:[3]

[INFO|(OpenDelta)basemodel:675]2022-07-23 02:56:49,063 >> Trainable Ratio: 100.000000%
[INFO|(OpenDelta)basemodel:677]2022-07-23 02:56:49,066 >> Delta Parameter Ratio: 0.164388%
[INFO|(OpenDelta)basemodel:679]2022-07-23 02:56:49,071 >> Static Memory 0.00 GB, Max Memory 0.00 GB


In [5]:
# a seperate example using BERT.
from transformers import BertForMaskedLM
from opendelta import AdapterModel
model = BertForMaskedLM.from_pretrained("bert-base-cased")
delta_model = AdapterModel(model) # This will apply adapter to the self-attn and feed-forward layer.
delta_model.log()

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
[INFO|(OpenDelta)structure_mapping:336]2022-07-23 02:59:07,427 >> Since you are using the common structure mapping, draw the transformed parameter structure for checking.


root
├── embeddings (Embedding) weight:[28996, 768]
├── bert.embeddings.position_embeddings (Embedding) weight:[512, 768]
├── bert.embeddings.token_type_embeddings (Embedding) weight:[2, 768]
├── bert.embeddings.LayerNorm (LayerNorm) weight:[768] bias:[768]
├── encoder (BertEncoder)
│   └── block (ModuleList)
│       └── 0-11(BertLayer)
│           ├── attn (BertAttention)
│           │   ├── q,k,v,proj(Linear) weight:[768, 768] bias:[768]
│           │   └── layer_norm (LayerNorm) weight:[768] bias:[768]
│           └── ff (BertOutput)
│               ├── w1 (Linear) weight:[3072, 768] bias:[3072]
│               ├── w2 (Linear) weight:[768, 3072] bias:[768]
│               └── layer_norm (LayerNorm) weight:[768] bias:[768]
└── lm_head (BertLMPredictionHead) bias:[28996]
    ├── transform.dense (Linear) weight:[768, 768] bias:[768]
    ├── transform.LayerNorm (LayerNorm) weight:[768] bias:[768]
    └── proj (Linear) weight:[28996, 768] bias:[28996]

root
├── bert (BertModel)
│   ├── embeddings (BertEmbeddings)
│   │   ├── word_embeddings (Embedding) weight:[28996, 768]
│   │   ├── position_embeddings (Embedding) weight:[512, 768]
│   │   ├── token_type_embeddings (Embedding) weight:[2, 768]
│   │   └── LayerNorm (LayerNorm) weight:[768] bias:[768]
│   └── encoder (BertEncoder)
│       └── layer (ModuleList)
│           └── 0-11(BertLayer)
│               ├── attention (BertAttention)
│               │   ├── self (BertSelfAttention)
│               │   │   └── query,key,value(Linear) weight:[768, 768] bias:[768]
│               │   ├── output (BertSelfOutput)
│               │   │   ├── dense (Linear) weight:[768, 768] bias:[768]
│               │   │   └── LayerNorm (LayerNorm) weight:[768] bias:[768]
│               │   └── adapter (AdapterLayer)
│               │       └── modulelist (Sequential)
│               │           ├── down_proj (Linear) weight:[24, 768] bias:[24]
│               │           └── up_proj (Linear) weight:[768, 24] bias:[768]
│               ├── intermediate (BertIntermediate)
│               │   └── dense (Linear) weight:[3072, 768] bias:[3072]
│               └── output (BertOutput)
│                   ├── dense (Linear) weight:[768, 3072] bias:[768]
│                   ├── LayerNorm (LayerNorm) weight:[768] bias:[768]
│                   └── adapter (AdapterLayer)
│                       └── modulelist (Sequential)
│                           ├── down_proj (Linear) weight:[24, 768] bias:[24]
│                           └── up_proj (Linear) weight:[768, 24] bias:[768]
└── cls (BertOnlyMLMHead)
    └── predictions (BertLMPredictionHead) bias:[28996]
        ├── transform (BertPredictionHeadTransform)
        │   ├── dense (Linear) weight:[768, 768] bias:[768]
        │   └── LayerNorm (LayerNorm) weight:[768] bias:[768]
        └── decoder (Linear) weight:[28996, 768] bias:[28996]

[INFO|(OpenDelta)basemodel:675]2022-07-23 02:59:07,689 >> Trainable Ratio: 100.000000%
[INFO|(OpenDelta)basemodel:677]2022-07-23 02:59:07,693 >> Delta Parameter Ratio: 0.827267%
[INFO|(OpenDelta)basemodel:679]2022-07-23 02:59:07,695 >> Static Memory 0.00 GB, Max Memory 0.00 GB


In [6]:
# continue with the BART example
delta_model.freeze_module(exclude=["deltas", "layernorm_embedding"], set_state_dict=True)
delta_model.log()

root
├── bert (BertModel)
│   ├── embeddings (BertEmbeddings)
│   │   ├── word_embeddings (Embedding) weight:[28996, 768]
│   │   ├── position_embeddings (Embedding) weight:[512, 768]
│   │   ├── token_type_embeddings (Embedding) weight:[2, 768]
│   │   └── LayerNorm (LayerNorm) weight:[768] bias:[768]
│   └── encoder (BertEncoder)
│       └── layer (ModuleList)
│           └── 0-11(BertLayer)
│               ├── attention (BertAttention)
│               │   ├── self (BertSelfAttention)
│               │   │   └── query,key,value(Linear) weight:[768, 768] bias:[768]
│               │   ├── output (BertSelfOutput)
│               │   │   ├── dense (Linear) weight:[768, 768] bias:[768]
│               │   │   └── LayerNorm (LayerNorm) weight:[768] bias:[768]
│               │   └── adapter (AdapterLayer)
│               │       └── modulelist (Sequential)
│               │           ├── down_proj (Linear) weight:[24, 768] bias:[24]
│               │           └── up_proj (Linear) weight:[768, 24] bias:[768]
│               ├── intermediate (BertIntermediate)
│               │   └── dense (Linear) weight:[3072, 768] bias:[3072]
│               └── output (BertOutput)
│                   ├── dense (Linear) weight:[768, 3072] bias:[768]
│                   ├── LayerNorm (LayerNorm) weight:[768] bias:[768]
│                   └── adapter (AdapterLayer)
│                       └── modulelist (Sequential)
│                           ├── down_proj (Linear) weight:[24, 768] bias:[24]
│                           └── up_proj (Linear) weight:[768, 24] bias:[768]
└── cls (BertOnlyMLMHead)
    └── predictions (BertLMPredictionHead) bias:[28996]
        ├── transform (BertPredictionHeadTransform)
        │   ├── dense (Linear) weight:[768, 768] bias:[768]
        │   └── LayerNorm (LayerNorm) weight:[768] bias:[768]
        └── decoder (Linear) weight:[28996, 768] bias:[28996]

[INFO|(OpenDelta)basemodel:675]2022-07-23 03:12:04,078 >> Trainable Ratio: 0.827267%
[INFO|(OpenDelta)basemodel:677]2022-07-23 03:12:04,081 >> Delta Parameter Ratio: 0.827267%
[INFO|(OpenDelta)basemodel:679]2022-07-23 03:12:04,084 >> Static Memory 0.00 GB, Max Memory 0.00 GB
